This notebook takes a look at the scrapped results from the BoardDocs website and gets the correct addresses for each row.

Input:
- `prelim_results.csv`

In [5893]:
import pandas as pd

df = pd.read_csv("prelim_results.csv")
df.head()

,URL,title_1,title_2,home_website
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,Board Policies and Guidelines,St. Joseph Public Schools,https://www.sjschools.org/
1,https://go.boarddocs.com/pa/cali/Board.nsf/Public,School Board Policy Manual,NaN,www.calsd.org
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",http://www.mapleschools.com
3,https://go.boarddocs.com/oh/rlsd/Board.nsf/Public,"585 Riverside Drive | Painesville, Ohio 44077 | 440.352.0668 | f 440.639.1959",Riverside Local School District,https://www.riversidelocalschools.com/
4,https://go.boarddocs.com/pa/shun/Board.nsf/Public,School Board Policy Manual,Southern Huntingdon County School District,http://www.shcsd.org


In [5894]:
# we remove redundant info
df.loc[df["title_1"]==df["title_2"], "title_1"] = None

In [5895]:
# trim whitespace
df.loc[:, "title_1"] = df["title_1"].str.strip()
df.loc[:, "title_2"] = df["title_2"].str.strip()

In [5896]:
# consider the 5-digit zip code approach
# check which rows have 5-digit codes in Title1, and those with them in Title2.
# Hopefully this will be a partition

# Define a regex pattern to match a 5-digit zip code
zip_code_pattern = r'\b\d{5}\b'

# Find rows where Title1 has a 5-digit zip code
df['Title1_has_zipcode'] = df['title_1'].str.contains(zip_code_pattern, na=False)

# Find rows where Title2 has a 5-digit zip code
df['Title2_has_zipcode'] = df['title_2'].str.contains(zip_code_pattern, na=False)

In [5897]:
# check if it is a partition
# first check if they add up

import numpy as np

print(f"Num of rows where title 1 has zipcode {df['Title1_has_zipcode'].sum()}")
print(f"Num of rows where title 2 has zipcode {df['Title2_has_zipcode'].sum()}")
print(f"Num of rows where title 1 or title 2 has zipcode {np.sum(df['Title1_has_zipcode'] | df['Title2_has_zipcode'])}")
print(f"Num of rows where title 1 and title 2 have zipcode {np.sum(df['Title1_has_zipcode'] & df['Title2_has_zipcode'])}")
print(f"Num of total rows {df.shape[0]}")

Num of rows where title 1 has zipcode 1440
Num of rows where title 2 has zipcode 1442
Num of rows where title 1 or title 2 has zipcode 2882
Num of rows where title 1 and title 2 have zipcode 0
Num of total rows 3906


In [5898]:
# ok great, there are no rows where you can find zipcodes on both cols
# but some rows don't have zipcodes in either
# let's check them out

no_zipcode_df = df[~(df['Title1_has_zipcode'] | df['Title2_has_zipcode'])]
print(f"Num of rows without zipcode {no_zipcode_df.shape[0]}")
no_zipcode_df.sample(5)

Num of rows without zipcode 1024


,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
3844,https://go.boarddocs.com/wi/westby/Board.nsf/Public,BoardDocs PL,,www.westby.k12.wi.us,False,False
3433,https://go.boarddocs.com/oh/aascoh/Board.nsf/Public,AACS Board of Education,Board Agendas,http://aacs.net/index.cfm,False,False
54,https://go.boarddocs.com/mi/rogers/Board.nsf/Public,Rogers City Area Schools,NEOLA Board of Education Policy Manual,http://www.rcashurons.org,False,False
3481,https://go.boarddocs.com/pa/npot/Board.nsf/Public,School Board Policy Manual,Northern Potter School District,http://www.northernpottersd.org/,False,False
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,False,False


In [5899]:
# let's get the proportions


print(f"Percentage of rows where title 1 has zipcode {df['Title1_has_zipcode'].sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 2 has zipcode {df['Title2_has_zipcode'].sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 1 or title 2 has zipcode {np.sum(df['Title1_has_zipcode'] | df['Title2_has_zipcode']).sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 1 and title 2 have zipcode {np.sum(df['Title1_has_zipcode'] & df['Title2_has_zipcode']).sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows with no zipcodes {no_zipcode_df.shape[0]/df.shape[0]*100:.2f}%")


Percentage of rows where title 1 has zipcode 36.87%
Percentage of rows where title 2 has zipcode 36.92%
Percentage of rows where title 1 or title 2 has zipcode 73.78%
Percentage of rows where title 1 and title 2 have zipcode 0.00%
Percentage of rows with no zipcodes 26.22%


In [5900]:
# some NaNs, some "Policy Manual", "School Board Policy and Guidelines", etc
# let's check the most common values

no_zipcode_df["title_1"].value_counts().head()

title_1
School Board Policy Manual              115
Policy Manual                            54
School Board Policies and Guidelines     41
School Board Policies                    22
BoardDocs PL                              9
Name: count, dtype: int64

In [5901]:
no_zipcode_df["title_2"].value_counts().head()

title_2
Board Policies                 24
                               22
School Board Policies          17
Board of Education             11
Board of Education Policies    11
Name: count, dtype: int64

In [5902]:
# ok, now let's check the website col
sum(df["home_website"].isna())

53

In [5903]:
# ok, unfortunately there are boarddocs without the home website linked
print(f"Percent of websites without links to official: {sum(df["home_website"].isna())/df.shape[0]*100:.3}%")

Percent of websites without links to official: 1.36%


In [5904]:
# but thankfully this number is small
# let's take a look at these websites

# do not truncate the col values in display
pd.set_option('display.max_colwidth', None)

df[df["home_website"].isna()].head()

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
232,https://go.boarddocs.com/pa/camb/Board.nsf/Public,NaN,NaN,NaN,False,False
443,https://go.boarddocs.com/oh/oakhil/Board.nsf/Public,NaN,NaN,NaN,False,False
465,https://go.boarddocs.com/oh/warrenoh/Board.nsf/Public,NaN,NaN,NaN,False,False
468,https://go.boarddocs.com/pa/marp/Board.nsf/Public,School Board Policy Manual,Marple Newtown School District,NaN,False,False
533,https://go.boarddocs.com/oh/meigs/Board.nsf/Public,NaN,NaN,NaN,False,False


In [5905]:
# after inspecting a few, it seems like they will usually write their school district as the h1 tag at least.

In [5906]:
# back to the address
# an observation is that I don't think those that don't contain zip codes will have addresses on the website
# let's check if a single number exists in them

number_pattern = r'\d'
print("Number of no-zipcode rows that contain a number")
sum(no_zipcode_df["title_1"].str.contains(number_pattern, na=False))

Number of no-zipcode rows that contain a number


46

In [5907]:
no_zipcode_df[no_zipcode_df["title_1"].str.contains(number_pattern, na=False)].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,False,False
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,POLICY,http://www.hartlandschools.us,False,False
3726,https://go.boarddocs.com/il/gsd34/Board.nsf/Public,Glenview School District 34,Glenview SD34,http://www.glenview34.org/,False,False
744,https://go.boarddocs.com/mo/lpsdmo/Board.nsf/Public,Liberty 53 School District,NaN,http://lps53.org,False,False
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",Board of Education,http://www.adc.d211.org,False,False


In [5908]:
no_zipcode_df[no_zipcode_df["title_2"].str.contains(number_pattern, na=False)].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,False,False
1203,https://go.boarddocs.com/in/brownsburg/Board.nsf/Public,Brownsburg Community School Corporation,"310 Stadium Drive Brownsburg, IN",https://www.brownsburg.k12.in.us/,False,False
1168,https://go.boarddocs.com/sd/custer/Board.nsf/Public,,Custer School District 16-1,http://www.csd.k12.sd.us/,False,False
380,https://go.boarddocs.com/wy/wyscsd1/Board.nsf/Public,NaN,Sublette County School District #1,http://www.sub1.org,False,False
3731,https://go.boarddocs.com/id/cascade/Board.nsf/Public,None,Cascade School District 422,http://www.cascadeschools.org,False,False


In [5909]:
# it turns out that they could either be an address with a missing zip code, or phone numbers
# there are also misc cases

# let's get a conservative (high) bound on the number of addresses that we will miss

num_no_zipcode_with_number = no_zipcode_df[no_zipcode_df["title_1"].str.contains(number_pattern, na=False) | no_zipcode_df["title_2"].str.contains(number_pattern, na=False)].shape[0]
num_no_zipcode_with_number

105

In [5910]:
num_zipcode = df.shape[0] - no_zipcode_df.shape[0]

In [5911]:
print(f"Worst case proportion of addresses that we will miss {num_no_zipcode_with_number/(num_no_zipcode_with_number+num_zipcode)*100:.2}%")

Worst case proportion of addresses that we will miss 3.5%


In [5912]:
# we put the ones we know are correct at a new address field

df["address"] = None
df.loc[df['Title1_has_zipcode'], "address"] = df["title_1"]
df.loc[df['Title2_has_zipcode'], "address"] = df["title_2"]
df.sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address
736,https://go.boarddocs.com/oh/osnaoh/Board.nsf/Public,"310 Browning Street, East Canton, Ohio 44730",Osnaburg Local Schools,http://www.ecweb.sparcc.org,True,False,"310 Browning Street, East Canton, Ohio 44730"
910,https://go.boarddocs.com/ar/siloam/Board.nsf/Public,Siloam Springs School District,"847 SOUTH DOGWOOD SILOAM SPRINGS, AR 72761",https://www.siloamschools.com,False,True,"847 SOUTH DOGWOOD SILOAM SPRINGS, AR 72761"
2828,https://go.boarddocs.com/pa/foxc/Board.nsf/Public,Policy Manual,Fox Chapel Area School District,http://www.fcasd.edu,False,False,None
1926,https://go.boarddocs.com/in/msddcs/Board.nsf/Public,"5275 Kentucky Avenue | Indianapolis, IN 46221 | 317-856-5265",MSD of Decatur Township,http://www.decaturproud.org,True,False,"5275 Kentucky Avenue | Indianapolis, IN 46221 | 317-856-5265"
3447,https://go.boarddocs.com/mt/kalispell/Board.nsf/Public,Kalispell Public Schools,"233 1st Avenue East - Kalispell, MT 59901",https://www.sd5.k12.mt.us/,False,True,"233 1st Avenue East - Kalispell, MT 59901"


In [5913]:
# now, let's try to get the school district name.
# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

In [5914]:
# number of rows with schools in at least one col
df[df["title_1_has_school"] | df["title_2_has_school"]].shape[0]

3206

In [5915]:
# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

356

In [5916]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
272,https://go.boarddocs.com/pa/fern/Board.nsf/Public,School Board Policy Manual,Ferndale Area School District,http://www.fasdk12.org,False,False,None,True,True
3192,https://go.boarddocs.com/pa/ever/Board.nsf/Public,EVERETT AREA SCHOOL DISTRICT,School Board Policies & Administrative Regulations,www.everettasd.org,False,False,None,True,True
1080,https://go.boarddocs.com/mi/pent/Board.nsf/Public,School Board Policies & Guidelines,Pentwater Public Schools,http://www.pentwaterschools.net,False,False,None,True,True
3714,https://go.boarddocs.com/mi/glads/Board.nsf/Public,School Board By-Laws and Policies,Gladstone Area Schools,http://www.gladstoneschools.com,False,False,None,True,True
11,https://go.boarddocs.com/pa/stlh/Board.nsf/Public,School Board Policy Manual,Steelton-Highspire School District,http://www.shsd.k12.pa.us,False,False,None,True,True


In [5917]:
# there is boilerplate text like School Board Policies
# let's try to get the top few popular ones and remove them

df["title_1"].value_counts().head(10)

title_1
School Board Policy Manual              118
Policy Manual                            55
School Board Policies and Guidelines     41
School Board Policies                    22
BoardDocs PL                              9
Board Policy and Guidelines               8
Board Policies                            7
School Board Policies and Bylaws          7
School Board Policies & Bylaws            6
Board Policies and Guidelines             6
Name: count, dtype: int64

In [5918]:
df["title_2"].value_counts().head(10)

title_2
Board Policies                      25
                                    25
School Board Policies               18
Board of Education                  14
Board of Education Policies         12
Board of Education Policy Manual    10
NEOLA Board Policies                10
Policy Manual                        6
School Board Policy Manual           6
Board Policy Manual                  5
Name: count, dtype: int64

In [5919]:
# let's remove these

remove_title_1_list = df["title_1"].value_counts().head(10).index
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
Board Policy                          6
School District Policies              6
Board Policy Manual                   5
Board Policies and Bylaws             5
School Board Policies & Guidelines    4
eGovernance Site                      4
School Board Policy                   4
School District Policy Manual         3
Board Policies & Bylaws               3
Board Policy and Bylaws               3
Name: count, dtype: int64

In [5920]:
# let's remove these
remove_title_1_list = df["title_1"].value_counts().head(10).index
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
Board of Education Policies and Guidelines                                                  3
School Board Policy and Guidelines                                                          3
School Board Policies and Administrative Guidelines                                         3
                                                                                            3
Board of Education Policies                                                                 3
2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180    2
Bloomfield School District                                                                  2
1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500                           2
School Board Policy & Guidelines                                                            2
1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009           2
Name: count, dtype: int64

In [5921]:
# let's remove the first few
remove_title_1_list = df["title_1"].value_counts().head(5).index
remove_title_1_list

Index(['Board of Education Policies and Guidelines',
       'School Board Policy and Guidelines',
       'School Board Policies and Administrative Guidelines', '',
       'Board of Education Policies'],
      dtype='object', name='title_1')

In [5922]:
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180    2
1300 Sherman Street Ste 222 | Sturgis, SD  57785                                            2
Bloomfield School District                                                                  2
1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500                           2
School Board Policy & Guidelines                                                            2
1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009           2
Electronic Governance System                                                                2
Policy Manual and Agendas                                                                   2
Board of Education                                                                          2
60 Jefferson Street, Suite 3• Monticello, NY 12701 • 845-794-7700                           2
Name: count, dtype: int64

In [5923]:
# pick those
remove_title_1_list = [
    "School Board By-Laws and Policies",
    "School Board policies and guidelines",
    "Policy Manual and Agendas",
    "Board of Education Policies",
    "School Board Policy and Bylaws",
    "Board Policy and Bylaws",
    "Board Policies and By-Laws",
    "School Board Policies ",
    "Board of Education",
    "School Board Policies and Guidlines",
    "School Board Bylaws and Policies",
    "SCHOOL BOARD POLICIES",
    "Board Policies & Guidelines",
    "School Board Policy & Guidelines",
    "Electronic Governance System",
    "School Board Agendas and Minutes",
    "School Board Policy and Administrative Guidelines",
    "School District Policies / Policy Manual",
    "School Policy Manual",
    "School Board Meetings",
    "School Board Policies & Ad Guidelines",
    "School Board meetings, agendas and policies",
    "School Board Policies and By Laws",
    "School board policies and manuals",
    "Board of School Directors",
    "School Board Agendas, Minutes, Policies and Guidelines",
    "Board Approved School Policy",
    "School Board Policies & Administrative Guidelines",
    "School Board Policy",
    "School Board Agendas, Policies and Guidelines",
    "School Board Bylaws/Policies/Guidelines",
    "SchoolBoard Policy Manual",
    "School Board Policy and Guidlines",
    "School Board Policy",
    "Governing Board Policy",
    "Board policy and guidelines",
    "Policies & By Laws",
    "Board of Trustees Policies and Administrative Guidelines",
    "Board of Education Policy and Bylaws"
    
]
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
print(df["title_1"].value_counts().index[:10])

Index(['2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180',
       '200 Reid Street | Palatka, FL 32177 | (386) 329-0602',
       '60 Jefferson Street, Suite 3• Monticello, NY 12701 • 845-794-7700',
       'Bloomfield School District',
       '1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500',
       '1300 Sherman Street Ste 222 | Sturgis, SD  57785',
       '315 N. French Avenue | Arlington  WA 98223 | 360.618.6200 | f 360.618.6221',
       '1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009',
       'Cleveland Metropolitan School District',
       '6301 Springside Avenue | Downers Grove, IL 60516 | Ph: (630) 795-7100  | Fx: (630) 795-7199'],
      dtype='object', name='title_1')


In [5924]:
# do this for title_2
# let's remove these
# NOTE: NEOLOA Board Policies might provide coarse information on whether the school could be located

remove_title_2_list = df["title_2"].value_counts().head(10).index
remove_title_2_list

Index(['Board Policies', '', 'School Board Policies', 'Board of Education',
       'Board of Education Policies', 'Board of Education Policy Manual',
       'NEOLA Board Policies', 'Policy Manual', 'School Board Policy Manual',
       'Board Policy Manual'],
      dtype='object', name='title_2')

In [5925]:
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().head(10)

title_2
eGovernance Site                 4
Bylaws & Policies                4
Board of Education Policy        4
Board of Education Meetings      3
Board Policy                     3
Neola Board Policies             3
Board of Directors               3
Arlington Public Schools         3
Putnam County School District    2
BoardDocs - Meeting Agendas      2
Name: count, dtype: int64

In [5926]:
remove_title_2_list = df["title_2"].value_counts().head(4).index
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().head(10)

title_2
Arlington Public Schools                  3
Neola Board Policies                      3
Board of Directors                        3
Board Policy                              3
Board Agendas                             2
Monticello Central School District        2
Meetings, Agendas and Information         2
Green Local Schools                       2
Santa Clara County Office of Education    2
Meade County, South Dakota                2
Name: count, dtype: int64

In [5927]:
remove_title_2_list = [
    "Meetings and Information",
    "Meetings, Agendas and Information",
    "Board Policy Manual",
    "Board of Education Meetings",
    "Board of Directors",
    "Policies",
    "BoardDocs - Meeting Agendas",
    "Board of Education Policies ",
    'Board Agendas',
    "NEOLA Board Policy",
    'NEOLA Board of Education Policies',
    'Neola Board Policies', 
    'NEOLA Board of Education Policy Manual',
    'NEOLA Policies', 
    'Meetings, Agendas, Information',
    'School Board Policies and Guidelines',
    "School Board Policies and Bylaws",
    "NEOLA School Board Policies",
    "Board of School Trustees Policy Manual",
    "School Board Policy	",
    "School Board Policies & Administrative Regulations	",
    "School Board Policy",
    "Board of School Trustees",
    "SCHOOL POLICIES AND GUIDELINES"
]
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().index[:20]

Index(['Arlington Public Schools', 'Board Policy',
       'Central Valley School District', 'Meade County, South Dakota',
       'Sumter District Schools', 'Santa Clara County Office of Education',
       'Green Local Schools', 'eGovernance System',
       'Monticello Central School District',
       'Community High School District 99', 'Putnam County School District',
       '8485 Homestead, Zeeland, MI 49464 Phone: 616-748-5637',
       'Iowa City Community School District',
       '2045 School Street North Collins, NY 14111', 'Barry ISD',
       'Adena Local Schools',
       '801 Corporate Centre Drive | O'Fallon, MO 63368 | Phone: 636-851-4000',
       '1323 E. 7th Street, Lockport, IL 60441',
       'Phone: 330-627-2181, - Fax: 330-627-2182',
       'Johnson County School District #1'],
      dtype='object', name='title_2')

In [5928]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

89

In [5929]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
2568,https://go.boarddocs.com/wi/sshore/Board.nsf/Public,South Shore School District,"9135 School Rd, Port Wing, WI 54865",https://sshore.org/,False,True,"9135 School Rd, Port Wing, WI 54865",True,True
2524,https://go.boarddocs.com/pa/elklake/Board.nsf/Public,"2380 Elk Lake School Road, Springville, PA 18844 | p 570-278-1106 | f 570-278-4838",Elk Lake School District and Susq. Co. Career & Tech. Center,http://www.elklakeschool.org/,True,False,"2380 Elk Lake School Road, Springville, PA 18844 | p 570-278-1106 | f 570-278-4838",True,True
2556,https://go.boarddocs.com/mo/rpsd/Board.nsf/Public,"21005 S School Rd. Peculiar, MO 64078 Phone: (816) 892-1300",Raymore-Peculiar School District,http://www.raypec.k12.mo.us,True,False,"21005 S School Rd. Peculiar, MO 64078 Phone: (816) 892-1300",True,True
3281,https://go.boarddocs.com/pa/naza/Board.nsf/Public,NASD Board of School Directors Meeting,Nazareth Area School District,http://www.nazarethasd.org,False,False,None,True,True
2211,https://go.boarddocs.com/pa/smet/Board.nsf/Public,Smethport Area School District,"414 South Mechanic Street | Smethport, PA 16749 | p-(814) 887-5543 | f-(814) 887-5544 | e-sasd@smethportschools.com",http://www.smethportschools.com/,False,True,"414 South Mechanic Street | Smethport, PA 16749 | p-(814) 887-5543 | f-(814) 887-5544 | e-sasd@smethportschools.com",True,True


In [5930]:
# I asked ChatGPT to inspect and find more titles I can remove

remove_title_1_list = [
    "School Board Agendas, Minutes, and Policies",
    "School Board Policies, Bylaws, and Guidelines",
    "School Policies and Guidelines",
    "School Board Policy Manual",
    "Board of School Trustees Policy",
    "School Board Agendas and Policies",
    "SchoolBoard Policy Manual",
    "School Board Agendas, Policies, Rules and Exhibits",
    "School Board Policy",
    "SchoolBoard Policy Manual ",
    "School District Policies ",
    "Board Meetings and Policies",
    "Board Management System",
    "Board of Education Policy and Guidelines",
    "Board Policy and By-Laws",
    "Policy and Bylaws",
    "Board Bylaws & Policies",
    "Bylaws & Policies",
    "Board of Trustees Policies",
    "Policies - Bylaws",
    "Bylaws and Policies",
    "NEOLA Policies",
    "Board of Education Bylaws and Policies",
    "School Board Policy Manual",
    "Board Administrative Guidelines , Bylaws, Forms and Policies",
    "Board Policies, Administrative Guidelines, and Forms",
    "Board Policies, Bylaws, Administrative Guidelines, Forms",
    "Board Agendas and Policies",
    "Policies of the Board of Education",
    "School Board Policies and Guidelines",
    "Corporation Board Policies and Guidelines",
    "NEOLA Board Policies and By-Laws",
    "Policies & Bylaws",
    "Board of Education Policies and Guidelines",
    "Policies - Bylaws",
    "Policies of the Board of Education",
    "Board Policy Handbook",
    "Board of Education Policies",
    "Board Bylaws and Policies",
    "Board Policy Manual and Administrative Guidelines",
    "Board Policies and Administrative Guidelines",
    "Board of Education NEOLA Policy",
    "Board of Education Policies and Administrative Guidelines",
    "Board of School Trustees Policy Manual",
    "School Board Policies and Guidelines",
    "Board Policy",
    "Policies and Administrative Guidelines",
    "Board of Education Policy and Administrative Guidelines",
    "Policies & Administrative Guidelines",
    "Meetings, Agendas, Policy Manual",
    "Joint Operating Committee (JOC) Policy Manual",
    "Board Policy & Guidelines",
    "BOARD OF TRUSTEES",
    "School Board Policies, Meeting Agendas and Minutes",
    "Board Policies and By-laws",
    "Board of Regents",
    "Board of Education Policies and By-Laws",
    "Board of Education Bylaws and Policies/Administrative Guidelines",
    "BoardDocs LT",
    "Fairless District Policy Manual",
    "Board of Education Bylaws & Policies",
    "Board Policies, Administrative Guidelines and Forms",
    "Board Policy & Bylaws",
    "Success for all in the 21st Century . . ."
]

df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None

In [5931]:
# I asked ChatGPT to inspect and find more titles I can remove

remove_title_2_list = [
    "POLICY",
    "Neola Board Policies & Guidelines",
    "Board of Education Policy and Guidelines",
    "NEOLA Policy Manual",
    "Board Policy",
    "Board Policies and Guidelines",
    "Policies And Administrative Guidelines",
    "Board Policies, Administrative Guidelines, and Forms",
    "Board of Education Policy",
    "Board Bylaws and Policies",
    "Policies - Bylaws",
    "Board of Education Bylaws and Policies",
    "Neola Board Policy & Administrative Guidelines",
    "Board Policy Handbook",
    "Policies of the Board of Education",
    "NEOLA Board Policy Manual",
    "Policies & Administrative Guidelines",
    "Policies & Bylaws",
    "Board Bylaws and Policies",
    "Board Policies",
    "Board of Education Policies",
    "Board of Education Policy and Administrative Guidelines",
    "Board Policy Manual",
    "Board of Education Policies and Administrative Guidelines",
    "BoardDocs LT",
    "Policies - Bylaws",
    "Policies of the Board of Education",
    "Board Policy and By-Laws",
    "Policies and Administrative Guidelines",
    "Board of Education Bylaws & Policies",
    "Board Policy",
    "Policies & Administrative Guidelines",
    "Board of Education School Policies",
    "School Board",
    "School Board Policies & Administrative Regulations",
    "School Board Policy",
    "Board of School Trustees Policy Manual",
    "School Board Policies and Guidelines",
    "School Board Policy ",
    "Board Policy",
    "Bylaws and Policies",
    "Meetings, Agenda and Information",
    "eGovernance System",
    "Meeting Packets",
    "Opportunity. Equity. Social Justice.",
    "Neola Board Policies & Guidelines",
    "Board Of Education Policy",
    "Board of Education NEOLA Policy",
    "Board Of Education Policies",
    "NEOLA - Board of Education Policies",
    "Board Policy Manual and Administrative Guidelines",
    "Policies & By-Laws",
    "Providing today's students opportunities to become tomorrow's leaders",
    "Board of School Trustees Policy",
    "Corporation Policies",
    "Board of Education Neola Polcies",
    "NEOLA Board Policies and By-Laws",
    "Board of Education Policy /Administrative Guidelines",
    "Board of Education Policies and Bylaws",
    "Board of Education Policies and Guidelines",
    "Neola Board of Education Policy Manual",
    "Neola - Board Policy",
    "Neola Board Policy",
    "Creating the Greatest Opportunities for Our Students"
]

df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None

In [5932]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

79

In [5933]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
704,https://go.boarddocs.com/nc/rock/Board.nsf/Public,"Rockingham County Schools | 511 Harrington Highway | Eden, NC 27288 | 336.627.2600",Rockingham County Schools,http://www.rock.k12.nc.us,True,False,"Rockingham County Schools | 511 Harrington Highway | Eden, NC 27288 | 336.627.2600",True,True
701,https://go.boarddocs.com/ny/letchworth/Board.nsf/Public,Letchworth Central School District,"5550 School Road Gainesville, NY 14066",https://www.letchworth.k12.ny.us/,False,True,"5550 School Road Gainesville, NY 14066",True,True
1652,https://go.boarddocs.com/wa/orcasisland/Board.nsf/Public,Orcas Island School Board,"557 School Road, Eastsound, Wa 98245 | Phone 360-376-2284",http://orcasislandschools.org/,False,True,"557 School Road, Eastsound, Wa 98245 | Phone 360-376-2284",True,True
3124,https://go.boarddocs.com/pa/nleb/Board.nsf/Public,346 School Drive,Northern Lebanon School District,www.norleb.org,False,False,None,True,True
3565,https://go.boarddocs.com/nh/rivendell/Board.nsf/Public,Rivendell School District,PO Box 271 10 School Drive Orford NH 03777,https://www.rivendellschool.org/,False,True,PO Box 271 10 School Drive Orford NH 03777,True,True


In [5934]:
# since a lot of it is from addresses,
# I will remove those where we have ported to the address field

df.loc[df["title_1"] == df["address"], "title_1"] = None
df.loc[df["title_2"] == df["address"], "title_2"] = None

In [5935]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

20

In [5936]:
# from 97 to 38, pretty good

# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
1578,https://go.boarddocs.com/oh/amanda/Board.nsf/Public,Amanda-Clearcreek Local Schools,Amanda-Clearcreek Local School District,http://www.amanda.k12.oh.us/,False,False,None,True,True
2366,https://go.boarddocs.com/oh/naps/Board.nsf/Public,www.napoleonareaschools.org | 419-599-7015,Napoleon Area City School District,www.napoleon.k12.oh.us,False,False,None,True,True
3124,https://go.boarddocs.com/pa/nleb/Board.nsf/Public,346 School Drive,Northern Lebanon School District,www.norleb.org,False,False,None,True,True
2031,https://go.boarddocs.com/wi/vasd/Board.nsf/Public,Verona Area School District,Verona Area School District,http://www.verona.k12.wi.us,False,False,None,True,True
538,https://go.boarddocs.com/ny/trivc/Board.nsf/Public,Tri-Valley Central School District Board of Education,Tri-Valley Central School District,http://www.trivalleycsd.org/,False,False,None,True,True


In [5937]:
# now, let's check those with both titles still intact

n = df[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]
print(f"Number of rows with both cols intact: {n}")

Number of rows with both cols intact: 156


In [5938]:
# let's make a school district column
# only do those to rows where EXACTLY one column has the word school district

df["title_1_has_school_district"] = df["title_1"].str.contains("school district",case=False,na=False)
df["title_2_has_school_district"] = df["title_2"].str.contains("school district",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]].shape[0]

4

In [5939]:
# check out these with both
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]]

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school,title_1_has_school_district,title_2_has_school_district
538,https://go.boarddocs.com/ny/trivc/Board.nsf/Public,Tri-Valley Central School District Board of Education,Tri-Valley Central School District,http://www.trivalleycsd.org/,False,False,None,True,True,True,True
769,https://go.boarddocs.com/pa/epns/Board.nsf/Public,EAST PENNSBORO AREA SCHOOL DISTRICT,East Pennsboro Area School District,http://www.epasd.org,False,False,None,True,True,True,True
1593,https://go.boarddocs.com/oh/labr/Board.nsf/Public,Policies of the LaBrae Local School District,LaBrae Local School District,https://labrae.school,False,False,None,True,True,True,True
2031,https://go.boarddocs.com/wi/vasd/Board.nsf/Public,Verona Area School District,Verona Area School District,http://www.verona.k12.wi.us,False,False,None,True,True,True,True


In [5940]:
# we can remove all in title_1
df.loc[df["title_1_has_school_district"] & df["title_2_has_school_district"], "title_1"] = None

In [5941]:
# let's make a school district column
# only do those to rows where EXACTLY one column has the word school district

df["title_1_has_school_district"] = df["title_1"].str.contains("school district",case=False,na=False)
df["title_2_has_school_district"] = df["title_2"].str.contains("school district",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]].shape[0]

0

In [5942]:
# now we can assign the school_district column

df.loc[df["title_1_has_school_district"],"school_district"] = df["title_1"]
df.loc[df["title_1_has_school_district"],"title_1"] = None

df.loc[df["title_2_has_school_district"],"school_district"] = df["title_2"]
df.loc[df["title_2_has_school_district"],"title_2"] = None

In [5943]:
# now let's look at the remaining columns
# first check those where both cols are still intact

df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]

70

In [5944]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school,title_1_has_school_district,title_2_has_school_district,school_district
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,False,False,None,True,True,False,False,NaN
2892,https://go.boarddocs.com/mi/gda/Board.nsf/Public,Gerald Dawkins Academy,(616) 219-0027,https://dawkinsacademy.org/,False,False,None,False,False,False,False,NaN
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,False,False,None,False,False,False,False,NaN
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,False,False,None,False,False,False,False,NaN
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,False,False,None,True,False,False,False,NaN


In [5945]:
# now, let's check those with policy or policies in their titles

df["title_1_has_policy"] = df["title_1"].str.contains("policy",case=False,na=False) | df["title_1"].str.contains("policies",case=False,na=False)
df["title_2_has_policy"] = df["title_2"].str.contains("policy",case=False,na=False) | df["title_2"].str.contains("policies",case=False,na=False)

# number of rows with policy in any cols
df[df["title_1_has_policy"] | df["title_2_has_policy"]].shape[0]

50

In [5946]:
# declutter columns
df = df.loc[:,["URL","title_1","title_2","home_website","address","school_district"]]

In [5947]:
df.head()

,URL,title_1,title_2,home_website,address,school_district
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,None,St. Joseph Public Schools,https://www.sjschools.org/,None,NaN
1,https://go.boarddocs.com/pa/cali/Board.nsf/Public,None,NaN,www.calsd.org,None,NaN
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,None,http://www.mapleschools.com,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",NaN
3,https://go.boarddocs.com/oh/rlsd/Board.nsf/Public,None,None,https://www.riversidelocalschools.com/,"585 Riverside Drive | Painesville, Ohio 44077 | 440.352.0668 | f 440.639.1959",Riverside Local School District
4,https://go.boarddocs.com/pa/shun/Board.nsf/Public,None,None,http://www.shcsd.org,None,Southern Huntingdon County School District


In [5948]:
# let's try to common title_1 and title_2
# first standardize None and NaN

print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_1"])
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_1"].isna())

1    None
Name: title_1, dtype: object
1    True
Name: title_1, dtype: bool


In [5949]:
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_2"])
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_2"].isna())

1    NaN
Name: title_2, dtype: object
1    True
Name: title_2, dtype: bool


In [5950]:
# set them to None
df.loc[df["title_1"].isna(), "title_1"] = None
df.loc[df["title_2"].isna(), "title_2"] = None

In [5951]:
# check those with both not None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]

70

In [5952]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].sample(5)

,URL,title_1,title_2,home_website,address,school_district
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN
2781,https://go.boarddocs.com/ca/dlinorthcounty/Board.nsf/Public,Dual Language Immersion North County,PHONE: (760) 203-5140,https://www.dlinorthcounty.org/,None,NaN


In [5953]:
# get those with phone numbers out
import re

# Function to check if a string contains a phone number
def contains_phone_number(value):
    # Regular expression for phone numbers
    phone_pattern = re.compile(r'\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
    if pd.isnull(value):
        return False
    return bool(phone_pattern.search(str(value)))

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df['contains_phone_number_title_2'] = df['title_2'].apply(contains_phone_number)

In [5954]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_1"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
1112,https://go.boarddocs.com/la/pcpsb/Board.nsf/Public,"337 Napoleon Street ● New Roads, Louisiana ● p 225-638-8674 ● f 225-638-3237",Pointe Coupee Parish School System,http://www.pcpsb.net/,None,NaN,True,False
1576,https://go.boarddocs.com/vsba/fairfax/Board.nsf/Public,"8115 Gatehouse Road, Suite 5400 | Falls Church, VA | 571-423-1075",Fairfax County School Board,http://www.fcps.edu,None,NaN,True,False
1987,https://go.boarddocs.com/oh/nls/Board.nsf/Public,600 Lemoyne Rd | Northwood OH | P: (419) 691-3888 | F: (419) 697-2470,Northwood Local Schools,http://www.northwoodschools.org/site/default.aspx?PageID=1,None,NaN,True,False
2150,https://go.boarddocs.com/oh/polaris/Board.nsf/Public,"7285 Old Oak Blvd., | Middleburg Heights, OH | 440-891-7600",Polaris Career Center,http://www.polaris.edu/,None,NaN,True,False
3356,https://go.boarddocs.com/wa/cowa/Board.nsf/Public,City Council Chambers ~ 500 E. Main Street ~ 509-488-5686 ~ www.othellowa.gov,Othello Washington ~ City Council ~ Serving The Community,http://www.othellowa.gov,None,NaN,True,False


In [5955]:
# all the above are address
move_these = both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_1"]]["URL"]
df.loc[df["URL"].isin(move_these), "address"] = df[df["URL"].isin(move_these)]["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

In [5956]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_2"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
159,https://go.boarddocs.com/in/resc/Board.nsf/Public,Randolph Eastern School Corporation,7659644994,http://www.resc.k12.in.us,None,NaN,False,True
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN,False,True
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,None,NaN,False,True
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN,False,True
1608,https://go.boarddocs.com/oh/sclsd/Board.nsf/Public,South Central Local Schools,3305 Greenwich Angling Rd | 419-752-3815,http://www.south-central.org,None,NaN,False,True
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN,False,True
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN,False,True
2306,https://go.boarddocs.com/mi/clark/Board.nsf/Public,Clarkston Community Schools,248-623-5400,http://www.clarkston.k12.mi.us,None,NaN,False,True
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN,False,True
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,None,NaN,False,True


In [5957]:
# move those two that are address
move_these = ["https://go.boarddocs.com/oh/sclsd/Board.nsf/Public", "https://go.boarddocs.com/oh/sidn/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "address"] = df["title_2"]
df.loc[df["URL"].isin(move_these), "title_2"] = None

In [5958]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_2"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
159,https://go.boarddocs.com/in/resc/Board.nsf/Public,Randolph Eastern School Corporation,7659644994,http://www.resc.k12.in.us,None,NaN,False,True
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN,False,True
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,None,NaN,False,True
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN,False,True
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN,False,True
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN,False,True
2306,https://go.boarddocs.com/mi/clark/Board.nsf/Public,Clarkston Community Schools,248-623-5400,http://www.clarkston.k12.mi.us,None,NaN,False,True
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN,False,True
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,None,NaN,False,True
2683,https://go.boarddocs.com/mi/slps/Board.nsf/Public,Spring Lake Public Schools,616-846-5500,http://www.springlakeschools.org,None,NaN,False,True


In [5959]:
# move the others to phone numbers
df["phone"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["contains_phone_number_title_2"], "phone"] = df["title_2"]
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["contains_phone_number_title_2"], "title_2"] = None

In [5960]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,None,NaN,False,False,None
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
356,https://go.boarddocs.com/ca/mendocino/Board.nsf/Public,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",https://www.mendocino.edu,None,NaN,False,False,None
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None


In [5961]:
# check those with numbers
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
2878,https://go.boarddocs.com/pa/daup/Board.nsf/Public,"School Board Policy Manual 6001 Locust Lane, Harrisburg, PA",Dauphin County Technical School,http://www.dcts.org,None,NaN,False,False,None
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [5962]:
# edit the outlier
df.loc[df["URL"]=="https://go.boarddocs.com/pa/daup/Board.nsf/Public", "title_1"] = "6001 Locust Lane, Harrisburg, PA"

In [5963]:
# check those with numbers
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
2878,https://go.boarddocs.com/pa/daup/Board.nsf/Public,"6001 Locust Lane, Harrisburg, PA",Dauphin County Technical School,http://www.dcts.org,None,NaN,False,False,None
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [5964]:
# move these two that are addresses
move_these = ["https://go.boarddocs.com/pa/daup/Board.nsf/Public", "https://go.boarddocs.com/wi/chilsd/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "address"] = df["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

# check
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [5965]:
# move the second one to district and delete the two others
df.loc[df["URL"]=="https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public", "title_1"] = None

# check
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [5966]:
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False), "title_1"] = None

In [5967]:
# do the same for title_2
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,None,NaN,False,False,None
356,https://go.boarddocs.com/ca/mendocino/Board.nsf/Public,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",https://www.mendocino.edu,None,NaN,False,False,None
1203,https://go.boarddocs.com/in/brownsburg/Board.nsf/Public,Brownsburg Community School Corporation,"310 Stadium Drive Brownsburg, IN",https://www.brownsburg.k12.in.us/,None,NaN,False,False,None


In [5968]:
# move them to address
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False),"address"] = df["title_2"]
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False),"title_2"] = None

In [5969]:
# check the rest
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx,:]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None


In [5970]:
# check those with schools in one but not in the other
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_1"].str.contains("school",case=False,na=False) & ~df["title_2"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,None,NaN,False,False,None
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN,False,False,None
1980,https://go.boarddocs.com/mi/fwlv/Board.nsf/Public,Fowlerville Community Schools,Home of the Gladiators,http://www.fowlervilleschools.org,None,NaN,False,False,None
2047,https://go.boarddocs.com/ok/okcps/Board.nsf/Public,Oklahoma City Public Schools,Ignite Passion. Instill Pride.,https://go.boarddocs.com/ok/okcps/Board.nsf/,None,NaN,False,False,None
2108,https://go.boarddocs.com/oh/maplecc/Board.nsf/Public,SCHOOL POLICIES AND GUIDELINES,MAPLEWOOD CAREER CENTER,http://www.mwood.cc/,None,NaN,False,False,None
2301,https://go.boarddocs.com/vsba/louisa/Board.nsf/Public,Louisa County Public Schools,Learners' Community,http://www.lcps.k12.va.us/education/components/scrapbook/default.php?sectiondetailid=1308&PHPSESSID=00a5e9972c66fd8162a098aed2356931,None,NaN,False,False,None
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN,False,False,None
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,None,NaN,False,False,None
3126,https://go.boarddocs.com/oh/zville/Board.nsf/Public,Zanesville City Schools District,Home of the Blue Devils,http://www.zanesville.k12.oh.us,None,NaN,False,False,None


In [5971]:
# take out the outlier
df.loc[df["URL"] == "https://go.boarddocs.com/oh/maplecc/Board.nsf/Public", "title_1"] = None

In [5972]:
# check those with schools in one but not in the other
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_1"].str.contains("school",case=False,na=False) & ~df["title_2"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,None,NaN,False,False,None
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN,False,False,None
1980,https://go.boarddocs.com/mi/fwlv/Board.nsf/Public,Fowlerville Community Schools,Home of the Gladiators,http://www.fowlervilleschools.org,None,NaN,False,False,None
2047,https://go.boarddocs.com/ok/okcps/Board.nsf/Public,Oklahoma City Public Schools,Ignite Passion. Instill Pride.,https://go.boarddocs.com/ok/okcps/Board.nsf/,None,NaN,False,False,None
2301,https://go.boarddocs.com/vsba/louisa/Board.nsf/Public,Louisa County Public Schools,Learners' Community,http://www.lcps.k12.va.us/education/components/scrapbook/default.php?sectiondetailid=1308&PHPSESSID=00a5e9972c66fd8162a098aed2356931,None,NaN,False,False,None
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN,False,False,None
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,None,NaN,False,False,None
3126,https://go.boarddocs.com/oh/zville/Board.nsf/Public,Zanesville City Schools District,Home of the Blue Devils,http://www.zanesville.k12.oh.us,None,NaN,False,False,None
3164,https://go.boarddocs.com/mi/clio/Board.nsf/Public,Clio Area Schools,Home of the Mustangs,http://www.clioschools.org,None,NaN,False,False,None


In [5973]:
# all slogans in second col, delete
df.loc[idx, "title_2"] = None

In [5974]:
# do it for the second col
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_2"].str.contains("school",case=False,na=False) & ~df["title_1"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
3308,https://go.boarddocs.com/in/hses/Board.nsf/Public,For additional information click the HOUSE icon to view streamed/archived board meeting videos.,Hamilton Southeastern Schools,https://www.hseschools.org/meet-hse/board,None,NaN,False,False,None
3393,https://go.boarddocs.com/mo/nixa/Board.nsf/Public,Board of Education Meeting Information,Nixa Public Schools,http://www.nixapublicschools.net,None,NaN,False,False,None
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,None,NaN,False,False,None
3443,https://go.boarddocs.com/mi/tda/Board.nsf/Public,THE DEARBORN ACADEMY,Public Charter School,http://www.thedearbornacademy.org,None,NaN,False,False,None


In [5975]:
# edit the outlier
df.loc[df["URL"] == "https://go.boarddocs.com/mi/tda/Board.nsf/Public", "title_2"] = None
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_2"].str.contains("school",case=False,na=False) & ~df["title_1"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
3308,https://go.boarddocs.com/in/hses/Board.nsf/Public,For additional information click the HOUSE icon to view streamed/archived board meeting videos.,Hamilton Southeastern Schools,https://www.hseschools.org/meet-hse/board,None,NaN,False,False,None
3393,https://go.boarddocs.com/mo/nixa/Board.nsf/Public,Board of Education Meeting Information,Nixa Public Schools,http://www.nixapublicschools.net,None,NaN,False,False,None
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,None,NaN,False,False,None


In [5976]:
# delete the first col
df.loc[idx,"title_1"] = None

In [5977]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None


In [5978]:
# for those with the word policy, make it the first col
policy_pattern = r'polic(y|ies)'
idx = df["title_2"].str.contains(policy_pattern, case=False,na=False)
df.loc[idx, ["title_1", "title_2"]] = df.loc[idx,["title_2","title_1"]].values

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/52244175.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df["title_2"].str.contains(policy_pattern, case=False,na=False)


In [5979]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,ROD Board Policy,Ripley-Ohio-Dearborn Special Education Cooperative,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None


In [5980]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/2999244493.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False)]


,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,ROD Board Policy,Ripley-Ohio-Dearborn Special Education Cooperative,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
2720,https://go.boarddocs.com/mi/giresa/Board.nsf/Public,Board Policies and Bylaws,Gratiot-Isabella Regional Education Service District,http://www.giresd.net,None,NaN,False,False,None
3326,https://go.boarddocs.com/mi/trav/Board.nsf/Public,TCAPS School Board Policies & Guidelines,Traverse City Area Public Schools,http://www.tcaps.net/board,None,NaN,False,False,None
3407,https://go.boarddocs.com/oh/tructc/Board.nsf/Public,TCTC Board Policies,Trumbull Career and Technical Center,http://www.tctchome.com,None,NaN,False,False,None
3525,https://go.boarddocs.com/pa/vnang/Board.nsf/Public,Venango Technology Center Policy Manual,Venango Technology Center,http://www.vtc1.org,None,NaN,False,False,None


In [5981]:
# can remove title_1 for these
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False), "title_1"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/786511925.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False), "title_1"] = None


,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
2889,https://go.boarddocs.com/ut/uen/Board.nsf/Public,Utah Education and Telehealth Network,UETN Governing Board,http://www.uetn.org,None,NaN,False,False,None
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County Schools,Jefferson County School Board,https://www.jeffersonschools.net,None,NaN,False,False,None
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN,False,False,None
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,None,NaN,False,False,None
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,None,NaN,False,False,None


In [5982]:
# do the same for board
idx = df["title_2"].str.contains("board", case=False,na=False)
df.loc[idx, ["title_1", "title_2"]] = df.loc[idx,["title_2","title_1"]].values

In [5983]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains("board",case=False,na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City School Board,Falls Church City Public Schools,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2889,https://go.boarddocs.com/ut/uen/Board.nsf/Public,UETN Governing Board,Utah Education and Telehealth Network,http://www.uetn.org,None,NaN,False,False,None
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County School Board,Jefferson County Schools,https://www.jeffersonschools.net,None,NaN,False,False,None
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,None,NaN,False,False,None


In [5984]:
# can remove title_1 for these
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains("board",case=False,na=False), "title_1"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN,False,False,None
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,None,NaN,False,False,None


In [5985]:
# just do case by case
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/ca/sjccd/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/ca/hawking/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/mi/badax/Board.nsf/Public", "title_2"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/in/fcsc/Board.nsf/Public", "title_1"] = None

df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone


In [5986]:
# check those with both not None
n = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]
print(n)
assert n == 0

0


In [5987]:
# now we can combine both
df.loc[df["title_1"].isna(), "title_1"] = df["title_2"]
df = df.drop("title_2", axis=1)

In [5988]:
# see what is left
df.loc[~df["title_1"].isna()].shape[0]

1819

In [5989]:
df.loc[~df["title_1"].isna()].sample(5)

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1928,https://go.boarddocs.com/oh/circle/Board.nsf/Public,Circleville City Schools,http://www.circlevillecityschools.org/,None,NaN,False,False,None
3233,https://go.boarddocs.com/oh/pleasoh/Board.nsf/Public,Pleasant Local Schools,pleasantlocalschools.org,None,NaN,False,False,None
3216,https://go.boarddocs.com/ca/santarosa/Board.nsf/Public,Sonoma County Junior College District,http://www.santarosa.edu/,None,NaN,False,False,None
3254,https://go.boarddocs.com/wa/bsdwa/Board.nsf/Public,Bellingham Public Schools,http://www.bellinghamschools.org,"1985 Barkley Boulevard | Bellingham, WA 98226 | Phone: 360-676-6400",NaN,False,False,None
1484,https://go.boarddocs.com/oh/tricountycc/Board.nsf/Public,Tri-County Career Center,https://www.tricountyhightech.com/,"15676 State Rt 691 | Nelsonville, OH 45764 | 740-753-3511",NaN,False,False,None


In [5990]:
# check those with numbers
df.loc[df["title_1"].str.contains(number_pattern, na=False)].shape[0]

134

In [5991]:
df.loc[df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
38,https://go.boarddocs.com/mo/mcr1/Board.nsf/Public,Macon County R-1 Schools,http://www.macon.k12.mo.us/,"702 North Missouri  Macon, Missouri 63552  (660) 395-6164",NaN,False,False,None
49,https://go.boarddocs.com/ks/susdks/Board.nsf/Public,Seaman USD 345,http://www.seamanschools.org,"901 NW Lyman Road | Topeka, KS 66608 | Ph: (785) 575-8600",NaN,False,False,None
76,https://go.boarddocs.com/pa/iu11/Board.nsf/Public,Tuscarora Intermediate Unit 11,https://www.tiu11.org/,None,NaN,False,False,None
125,https://go.boarddocs.com/wi/pesh/Board.nsf/Public,341 NORTH EMERY AVENUE,www.peshtigo.k12.wi.us,None,PESHTIGO SCHOOL DISTRICT,False,False,None
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
...,...,...,...,...,...,...,...,...
3806,https://go.boarddocs.com/ks/usd315/Board.nsf/Public,Colby Public Schools USD 315,http://www.colbyeagles.org/,600 W 3rd St. | Colby KS 67701-2000 | p 785-460-5000 | f 785-460-5050,NaN,False,False,None
3816,https://go.boarddocs.com/fl/semi/Board.nsf/Public,Phone - 407-320-0000,www.scps.k12.fl.us,"400 E. Lake Mary Boulevard - Sanford, FL - 32773",NaN,False,True,None
3820,https://go.boarddocs.com/ks/usd311/Board.nsf/Public,Pretty Prairie USD 311 KS,https://www.usd311.com/,"206 E Main, P.O. Box 218 Pretty Prairie, Kansas 67570",NaN,False,False,None
3877,https://go.boarddocs.com/pa/neiu/Board.nsf/Public,Northeastern Educational Intermediate Unit 19,http://www.iu19.org,1200 Line Street Archbald PA 18403,NaN,False,False,None


In [5992]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]].shape[0]

38

In [5993]:
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
680,https://go.boarddocs.com/pa/prtg/Board.nsf/Public,(814) 736-9636,www.portageareasd.org,"84 Mountain Avenue, Portage, PA 15946",NaN,True,True,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,(906) 224-7211,http://www.wmschools.org,None,Wakefield-Marenisco School District,True,True,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None


In [5994]:
# move those without alphabets to the phone col

alphabet_pattern = r'[a-zA-Z]'
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)) ]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
680,https://go.boarddocs.com/pa/prtg/Board.nsf/Public,(814) 736-9636,www.portageareasd.org,"84 Mountain Avenue, Portage, PA 15946",NaN,True,True,None
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,(906) 224-7211,http://www.wmschools.org,None,Wakefield-Marenisco School District,True,True,None
1765,https://go.boarddocs.com/pa/iu14/Board.nsf/Public,610-987-2248,http://www.berksiu.org/,"1111 Commons Boulevard, PO Box 16050, 19612-6050",NaN,True,True,None
1924,https://go.boarddocs.com/wi/shorewood/Board.nsf/Public,(414) 963-6901,https://www.shorewood.k12.wi.us,"Shorewood School District | 1701 East Capitol Drive | Shorewood, Wisconsin 53211",NaN,True,True,None
1985,https://go.boarddocs.com/pa/roch/Board.nsf/Public,724-775-7500,http://www.rasd.org,"540 Reno Street, Rochester, PA 15074",NaN,True,True,None
2028,https://go.boarddocs.com/mi/fowler/Board.nsf/Public,(989)593-2250,http://www.fowlerschools.net,"700 S Main Street, Fowler MI 48835",NaN,True,True,None
2525,https://go.boarddocs.com/nj/burlingtontwp/Board.nsf/Public,(609) 387-3955,https://burltwpsch.org/,None,Burlington Township School District,True,True,None
2767,https://go.boarddocs.com/in/jcdc/Board.nsf/Public,812-689-4114,http://www.jaccendel.k12.in.us,"723 N Buckeye St, Osgood IN 47037",NaN,True,True,None
2874,https://go.boarddocs.com/wi/hfj1/Board.nsf/Public,(262) 673-3155,www.hjt1.org,"School District of Hartford Jt. #1, 402 W. Sumner St. Hartford, WI 53027",NaN,True,True,None
3159,https://go.boarddocs.com/wi/wawmsd/Board.nsf/Public,414-604-3000,http://www.wawmsd.org,"West Allis-West Milwaukee School District, 9333 W. Lincoln Avenue, West Allis, WI 53227",NaN,True,True,None


In [5995]:
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)), "phone"] = df["title_1"]
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)), "title_1"] = None


In [5996]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]].shape[0]

28

In [5997]:
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
1574,https://go.boarddocs.com/oh/swissohio/Board.nsf/Public,Phone: 740-472-5801,https://swissohio.k12.oh.us,"304 Mill Street Woodsfield, OH 43793",NaN,True,True,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None


In [5998]:
# the relevant info in the column is either phone, address or both
# the row with the website already has the website col filled
# first get those that are phone only

In [5999]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*[Pp](hone)[^a-zA-Z]*$'

# Filter the rows
df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/2831296269.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1574,https://go.boarddocs.com/oh/swissohio/Board.nsf/Public,Phone: 740-472-5801,https://swissohio.k12.oh.us,"304 Mill Street Woodsfield, OH 43793",NaN,True,True,None
2140,https://go.boarddocs.com/wi/solon/Board.nsf/Public,Phone: (715) 378-2263,https://www.solonk12.net,"School District of Solon Springs, 8993 E Baldwin Ave, Solon Springs, WI 54873",NaN,True,True,None
2160,https://go.boarddocs.com/oh/galionoh/Board.nsf/Public,Phone - 419-468-3432,https://www.galionschools.org/,"Galion City Schools - 470 Portland Way North - Galion, OH 44833",NaN,True,True,None
3816,https://go.boarddocs.com/fl/semi/Board.nsf/Public,Phone - 407-320-0000,www.scps.k12.fl.us,"400 E. Lake Mary Boulevard - Sanford, FL - 32773",NaN,True,True,None


In [6000]:
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/3918443988.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/3918443988.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None


In [6001]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,True,True,None


In [6002]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*([Pp]hone|PH|MAIN PHONE)?[^a-zA-Z]*([Ff]ax)?[^a-zA-Z]*$'

# Filter the rows
df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/3636070497.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
2538,https://go.boarddocs.com/wi/campsd/Board.nsf/Public,(920) 533-8381 | Fax (920) 533 -5726,www.csd.k12.wi.us,"327 N. Fond du Lac Ave. Campbellsport, WI 53010",NaN,True,True,None
2586,https://go.boarddocs.com/oh/cevsdoh/Board.nsf/Public,"Phone: 330-627-2181, - Fax: 330-627-2182",www.carrollton.k12.oh.us,"Carrollton Exempted Village School District, - 205 Scio Road S.W., - Carrollton, OH 44615",NaN,True,True,None
2811,https://go.boarddocs.com/id/nsd131/Board.nsf/Public,(208) 468-4600 Fax: (208) 468-4638,http://www.nsd131.org,"619 S. Canyon St | Nampa, ID 83686",NaN,True,True,None
3001,https://go.boarddocs.com/fl/highlfl/Board.nsf/Public,PH: 863-471-5555,www.highlands.k12.fl.us,"School Board of Highlands County, 426 School St., Sebring, FL 33870",NaN,True,True,None
3758,https://go.boarddocs.com/fl/brevco/Board.nsf/Public,* MAIN PHONE: (321) 633-1000,https://www.brevardschools.org/,"2700 JUDGE FRAN JAMIESON WAY, VIERA, FL 32940",NaN,True,True,None


In [6003]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*([Pp]hone|PH|MAIN PHONE)?[^a-zA-Z]*([Ff]ax)?[^a-zA-Z]*$'

# Filter the rows
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/464474225.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_41628/464474225.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None


In [6004]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,True,True,None
2366,https://go.boarddocs.com/oh/naps/Board.nsf/Public,www.napoleonareaschools.org | 419-599-7015,www.napoleon.k12.oh.us,None,Napoleon Area City School District,True,False,None


In [6005]:
# handle special cases
move_these = ["https://go.boarddocs.com/ks/usd230/Board.nsf/Public","https://go.boarddocs.com/oh/naps/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "phone"] = df["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

In [6006]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2495,https://go.boarddocs.com/in/brem/Board.nsf/Public,Bremen Public Schools | Phone: (574) 546-3929 | Fax: (574) 546-6303 | School Board Policies and Guidelines,https://www.bps.k12.in.us,None,NaN,True,False,None
2808,https://go.boarddocs.com/wi/ashland/Board.nsf/Public,"District Office - 2000 Beaser Avenue; Ashland, WI; (715) 682-7080",http://www.ashland.k12.wi.us/,None,School District of Ashland Board of Education,True,False,None


In [6007]:
# these are all addresses, move them
df.loc[df["contains_phone_number_title_1"], "address"] = df["title_1"]
df.loc[df["contains_phone_number_title_1"], "title_1"] = None

In [6008]:
# now check the remaining ones
print(df[~df["title_1"].isna()].shape[0])
df[~df["title_1"].isna()].sample(5)

1781


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1698,https://go.boarddocs.com/oh/delphos/Board.nsf/Public,Delphos City Schools,https://www.delphoscityschools.org/,"901 Wildcat Lane | Delphos, OH 45833 | 419-692-2509",NaN,False,False,None
2728,https://go.boarddocs.com/in/dccs/Board.nsf/Public,Decatur County Community Schools,http://www.decaturco.k12.in.us,None,NaN,False,False,None
3762,https://go.boarddocs.com/oh/sciville/Board.nsf/Public,Sciotoville Community Schools,https://www.easttartans.org/,"224 Marshall St, Portsmouth, Ohio 45662 | Phone: 7407766777",NaN,False,False,None
478,https://go.boarddocs.com/mi/tric/Board.nsf/Public,Tri County Area Schools,http://www.tricountyschools.com,None,NaN,False,False,None
99,https://go.boarddocs.com/oh/river/Board.nsf/Public,Riverdale Local Schools,www.riverdalefalcons.org,None,NaN,False,False,None


In [6009]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
901,https://go.boarddocs.com/nd/bsd7/Board.nsf/Public,"Agendas, Minutes, District Information, Policies and Events",http://www.belcourt.k12.nd.us/education/components/scrapbook/default.php?sectiondetailid=1376&,None,Turtle Mountain Community Schools/Belcourt School District #7,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
1736,https://go.boarddocs.com/pa/uncf/Board.nsf/Public,"Meetings, Agendas, and Information",http://www.ucfsd.org,None,Unionville-Chadds Ford School District,False,False,None
1800,https://go.boarddocs.com/pa/boyr/Board.nsf/Public,"Meetings, Agendas, Information",http://www.boyertownasd.org,None,NaN,False,False,None
2077,https://go.boarddocs.com/mo/unionrxi/Board.nsf/Public,"Meetings, Agendas, and Information",https://www.unionrxi.org,None,Union R-XI School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2521,https://go.boarddocs.com/oh/brightoh/Board.nsf/Public,"Meeting Agendas, Minutes and Policies",https://www.blsd.us,None,Bright Local School District,False,False,None


In [6010]:
# the agenda value keep coming up
# let's check for all rows on this
df.loc[df["title_1"].str.contains("agenda",case=False,na=False)]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
157,https://go.boarddocs.com/il/sd163/Board.nsf/Public,Board Meeting Agendas,http://www.sd163.com,None,NaN,False,False,None
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,Laurel School Board Agendas and Policy Manual,http://www.laurel.k12.pa.us,None,Laurel School District,False,False,None
901,https://go.boarddocs.com/nd/bsd7/Board.nsf/Public,"Agendas, Minutes, District Information, Policies and Events",http://www.belcourt.k12.nd.us/education/components/scrapbook/default.php?sectiondetailid=1376&,None,Turtle Mountain Community Schools/Belcourt School District #7,False,False,None
1163,https://go.boarddocs.com/ca/vibrantminds/Board.nsf/Public,"Board Agendas, Minutes, and Policies",https://www.vibrantminds.us/,"412 W. CARL KARCHER WAY ANAHEIM, CA 92801 | ​714-563-2390",NaN,False,False,None
1736,https://go.boarddocs.com/pa/uncf/Board.nsf/Public,"Meetings, Agendas, and Information",http://www.ucfsd.org,None,Unionville-Chadds Ford School District,False,False,None
1800,https://go.boarddocs.com/pa/boyr/Board.nsf/Public,"Meetings, Agendas, Information",http://www.boyertownasd.org,None,NaN,False,False,None
2077,https://go.boarddocs.com/mo/unionrxi/Board.nsf/Public,"Meetings, Agendas, and Information",https://www.unionrxi.org,None,Union R-XI School District,False,False,None
2472,https://go.boarddocs.com/pa/iu24/Board.nsf/Public,Board Agendas and Policy Manual,http://www.cciu.org,"455 Boot Road, Downingtown, PA 19335 | 484-237-5000",NaN,False,False,None
2521,https://go.boarddocs.com/oh/brightoh/Board.nsf/Public,"Meeting Agendas, Minutes and Policies",https://www.blsd.us,None,Bright Local School District,False,False,None
2646,https://go.boarddocs.com/oh/mayoh/Board.nsf/Public,Meeting Agendas & Minutes,http://www.mayfieldschools.org,None,Mayfield City School District,False,False,None


In [6011]:
# remove them

df.loc[df["title_1"].str.contains("agenda",case=False,na=False), "title_1"] = None

In [6012]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2627,https://go.boarddocs.com/nj/hhboe/Board.nsf/Public,"316-A Seventh Ave Haddon Heights, New Jersey",http://gogarnets.com/,None,Haddon Heights School District,False,False,None
2710,https://go.boarddocs.com/ak/swrsdak/Board.nsf/Public,"... educating our future, guided by our past",http://www.swrsd.org,None,Southwest Region School District,False,False,None
2729,https://go.boarddocs.com/ca/cvesd/Board.nsf/Public,"84 East J Street , Chula Vista , CA91910",https://www.cvesd.org/,None,Chula Vista Elementary School District,False,False,None
2824,https://go.boarddocs.com/mo/wpr7sd/Board.nsf/Public,"Excellence in Education, Service, Life.",https://www.zizzers.org,None,West Plains School District,False,False,None
3287,https://go.boarddocs.com/wi/afasd/Board.nsf/Public,"201 W. 6th Street, Friendship, WI",https://www.afasd.net/,None,ADAMS-FRIENDSHIP AREA SCHOOL DISTRICT,False,False,None


In [6013]:
# get those with numbers first

df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna() & df["title_1"].str.contains(number_pattern),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2627,https://go.boarddocs.com/nj/hhboe/Board.nsf/Public,"316-A Seventh Ave Haddon Heights, New Jersey",http://gogarnets.com/,None,Haddon Heights School District,False,False,None
2729,https://go.boarddocs.com/ca/cvesd/Board.nsf/Public,"84 East J Street , Chula Vista , CA91910",https://www.cvesd.org/,None,Chula Vista Elementary School District,False,False,None
3287,https://go.boarddocs.com/wi/afasd/Board.nsf/Public,"201 W. 6th Street, Friendship, WI",https://www.afasd.net/,None,ADAMS-FRIENDSHIP AREA SCHOOL DISTRICT,False,False,None


In [6014]:
# move them to the address
idx = df["title_1"].str.contains(",",na=False) & df["address"].isna() & df["title_1"].str.contains(number_pattern)
df.loc[idx,"address"] = df["title_1"]
df.loc[idx,"title_1"] = None

In [6015]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2710,https://go.boarddocs.com/ak/swrsdak/Board.nsf/Public,"... educating our future, guided by our past",http://www.swrsd.org,None,Southwest Region School District,False,False,None
2824,https://go.boarddocs.com/mo/wpr7sd/Board.nsf/Public,"Excellence in Education, Service, Life.",https://www.zizzers.org,None,West Plains School District,False,False,None


In [6016]:
# move the first one to address and remove the rest, which are slogans
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "title_1"] = None

df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),"title_1"] = None

In [6017]:
# check the remaining ones
print(df.loc[~df["title_1"].isna()].shape[0])
df.loc[~df["title_1"].isna()].sample(5)

1757


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1120,https://go.boarddocs.com/ny/manhps/Board.nsf/Public,Manhasset Public Schools NY,https://www.manhassetschools.org,"200 Memorial Place Manhasset, NY 11030 | 516-267-7700",NaN,False,False,None
2570,https://go.boarddocs.com/il/ccsd146/Board.nsf/Public,CCSD 146,http://www.district146.org,None,Community Consolidated School District #146,False,False,None
195,https://go.boarddocs.com/nj/shto/Board.nsf/Public,Shamong Township Schools,http://www.shamongschools.org/,"295 Indian Mills Road | Shamong, NJ 08088 | (609) 268-0316 | f (609) 268-1229",NaN,False,False,None
131,https://go.boarddocs.com/oh/bloomcarroll/Board.nsf/Public,Bloom-Carroll Local Schools,https://www.bloom-carroll.k12.oh.us/,"5240 Plum Road NW | Carroll, OH 43112 | 614-837-6560",NaN,False,False,None
717,https://go.boarddocs.com/ca/whccd/Board.nsf/Public,West Hills Community College District,http://www.westhillscollege.com/,"275 Phelps Avenue | Coalinga, CA 93210 | 559-934-2180",NaN,False,False,None


In [6018]:
# check those with address na too
print(df.loc[~df["title_1"].isna() & df["address"].isna()].shape[0])
df.loc[~df["title_1"].isna() & df["address"].isna()].sample(5)

559


/opt/miniconda3/envs/cepr_schoolboard/lib/python3.13/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
925,https://go.boarddocs.com/mi/brown/Board.nsf/Public,Brown City Community Schools,https://www.browncityschools.org,None,NaN,False,False,None
2237,https://go.boarddocs.com/wi/ashsd/Board.nsf/Public,A Tradition of Excellence & Opportunity,www.ashwaubenon.k12.wi.us,None,Ashwaubenon School District,False,False,None
3479,https://go.boarddocs.com/mi/cenlak/Board.nsf/Public,Central Lake Public Schools,http://www.centrallake.org,None,NaN,False,False,None
3269,https://go.boarddocs.com/mi/ubly/Board.nsf/Public,Ubly Community Schools,http://www.ublyschools.org,None,NaN,False,False,None
3500,https://go.boarddocs.com/mi/carain/Board.nsf/Public,Carman-Ainsworth Community Schools,http://www.carmanainsworth.org,None,NaN,False,True,810-591-3700


In [6019]:
# check those with school_district na too
print(df.loc[~df["title_1"].isna() & df["school_district"].isna()].shape[0])
df.loc[~df["title_1"].isna() & df["school_district"].isna()].sample(5)

1704


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
591,https://go.boarddocs.com/va/nucps/Board.nsf/Public,Northumberland County School Board Office,https://www.nucps.net/,"6958 Northumberland Highway, Heathsville, VA | P.O. Box 730, Heathsville, VA 22473 l phone 804.529.6134 | fax 804.529.6449 |",NaN,False,False,None
2784,https://go.boarddocs.com/ny/genevacsd/Board.nsf/Public,Geneva City Schools,http://www.genevacsd.org/,"400 West North Street | Geneva, NY 14456 | (315) 781-0400",NaN,False,False,None
140,https://go.boarddocs.com/oh/trccoh/Board.nsf/Public,Tri-Rivers Career Center,www.tririvers.com,None,NaN,False,False,None
867,https://go.boarddocs.com/oh/jaaga/Board.nsf/Public,James A. Garfield Local Schools,http://jagschools.org,None,NaN,False,False,None
1713,https://go.boarddocs.com/in/smont/Board.nsf/Public,Southmont Schools,http://www.southmontschools.org,None,NaN,False,False,None


In [6020]:
# check those with non-alphabets in title_1
non_alpha_pattern = r'[^A-Za-z ]'
print(df.loc[df["title_1"].str.contains(non_alpha_pattern, na=False)].shape[0])
df.loc[df["title_1"].str.contains(non_alpha_pattern, na=False)].sample(5)

319


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1662,https://go.boarddocs.com/tx/cobtx/Board.nsf/Public,"City of Bryan, Texas",https://www.bryantx.gov/,"300 S. Texas Ave. Bryan, TX 77803 (979) 209-5000",NaN,False,False,None
2772,https://go.boarddocs.com/ny/jlboces/Board.nsf/Public,Jefferson-Lewis BOCES,https://www.boces.com/,"20104 State Route 3, Watertown, NY 13601",NaN,False,False,None
1346,https://go.boarddocs.com/la/scpps/Board.nsf/Public,St. Charles Parish Public Schools,http://www.wearescpps.org,"13855 River Road * Luling , Louisiana 70070 * Ph: (985) 785-6289",NaN,False,False,None
2051,https://go.boarddocs.com/oh/blme/Board.nsf/Public,Bloomfield-Mespo Local Schools,https://www.bloomfieldmesposchools.org/,None,NaN,False,False,None
165,https://go.boarddocs.com/il/vobil/Board.nsf/Public,"The Village of Bourbonnais, Illinois",http://www.villageofbourbonnais.com/,"600 Main St N.W. | Bourbonnais, IL 60914",NaN,False,False,None


In [6021]:
# for those that have addresses set, we can move them to school_district
df.loc[~df["address"].isna() & ~df["title_1"].isna()]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,http://www.mapleschools.com,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",NaN,False,False,None
10,https://go.boarddocs.com/in/centergrove/Board.nsf/Public,Center Grove Community School Corporation,https://www.centergrove.k12.in.us/,"4800 West Stones Crossing Road | Greenwood, IN 46143 | (317) 881-9326",NaN,False,False,None
12,https://go.boarddocs.com/oh/pcc/Board.nsf/Public,Penta Career Center,http://www.pentacareercenter.org/,"9301 Buck Rd. | Perrysburg, Ohio 43551 | High School: 419-666-1120 Adult Education: 419-661-6555",NaN,False,False,None
15,https://go.boarddocs.com/oh/westholmes/Board.nsf/Public,West Holmes Local Schools,https://westholmes.org/,"28 W Jackson St. | Millersburg, OH 44654 | 330-674-3546",NaN,False,False,None
16,https://go.boarddocs.com/mabe/carps/Board.nsf/Public,Caroline County Public Schools,carolineschools.org,"Address: 204 Franklin Street | Denton, MD 21629 | Phone: (410) 479-1460",NaN,False,False,None
...,...,...,...,...,...,...,...,...
3897,https://go.boarddocs.com/co/jeffco/Board.nsf/Public,Jeffco Public Schools Board of Education,http://www.jeffcopublicschools.org/,1829 Denver West Drive | Golden. CO 80401 | (303) 982-6800,NaN,False,False,None
3902,https://go.boarddocs.com/il/cowil/Board.nsf/Public,City of Waukegan,http://www.waukeganil.gov/,"100 N. Martin Luther King Jr. Ave. | Waukegan, IL 60085",NaN,False,False,None
3903,https://go.boarddocs.com/ca/laccd/Board.nsf/Public,Los Angeles Community College District,http://laccd.edu,"770 Wilshire Boulevard, Los Angeles, CA 90017 | (213) 891-2000",NaN,False,False,None
3904,https://go.boarddocs.com/md/stmarysco/Board.nsf/Public,"St. Mary's County, Maryland",http://www.stmarysmd.com,"41770 Baldridge Street | Leonardtown, MD 20650 | 301-475-4200",NaN,False,False,None


In [6022]:
# check if there are any where all three are set
df.loc[~df["address"].isna() & ~df["title_1"].isna() & ~df["school_district"].isna()]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone


In [6023]:
# not really sure if it will be entirely correct
# check for those without address set
print(df.loc[df["address"].isna() & ~df["title_1"].isna()].shape[0])
df.loc[df["address"].isna() & ~df["title_1"].isna()].head()

559


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,St. Joseph Public Schools,https://www.sjschools.org/,None,NaN,False,False,None
6,https://go.boarddocs.com/pa/fcctc/Board.nsf/Public,Franklin County Career and Technology Center,www.franklinctc.com,None,NaN,False,False,None
9,https://go.boarddocs.com/ca/voc/Board.nsf/Public,Vista Oaks Charter School,https://www.vistaoaks.net,None,NaN,False,False,None
20,https://go.boarddocs.com/mi/man/Board.nsf/Public,Manton Consolidated Schools,http://mantonmi.apptegy.us/o/mcs?mode=edit,None,NaN,False,False,None
21,https://go.boarddocs.com/pa/cdsd/Board.nsf/Public,Quality to the Core,http://www.cdschools.org,None,Central Dauphin School District,False,False,None


In [6024]:
# not really sure if it will be entirely correct
# check for those with both unset
print(df.loc[df["address"].isna() & df["school_district"].isna() & ~df["title_1"].isna()].shape[0])
df.loc[df["address"].isna() & df["school_district"].isna() & ~df["title_1"].isna()].sample(5)

506


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
3784,https://go.boarddocs.com/wv/morg/Board.nsf/Public,Morgan County Schools,http://www.morganschools.net,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,http://www.csredhawks.org,None,NaN,False,False,None
3727,https://go.boarddocs.com/pa/cccc/Board.nsf/Public,Clarion County Career Center,http://www.clarioncte.org,None,NaN,False,False,None
3407,https://go.boarddocs.com/oh/tructc/Board.nsf/Public,Trumbull Career and Technical Center,http://www.tctchome.com,None,NaN,False,False,None
923,https://go.boarddocs.com/oh/ytwnsd/Board.nsf/Public,Youngstown Community School,http://www.youngstowncommunityschool.k12.oh.us/,None,NaN,False,False,None


In [6025]:
# check how many we have left
sum(~df["title_1"].isna())

1757

In [6026]:
# send to GPT to deal with it
df.to_csv("test.csv", columns=["title_1","address","school_district"], index=False)

In [6027]:
# I gave GPT-4o the following prompt
"""
This csv has three columns: title_1, address, and school_district. Many values are empty.

Your goal is to move the value from title_1 to either address, or school_district, or simply discard it. Do not overwrite any values.

For example, for the row

Christopher Newport University,"1 Avenue of the Arts Newport News, VA 23606 | (757) 594-7000",

You should put Christopher Newport University as the school_district, which is third column, that is

,"1 Avenue of the Arts Newport News, VA 23606 | (757) 594-7000","Christopher Newport University"

Another example

Quality to the Core,,Central Dauphin School District

You should just drop the title_1, since it is a slogan

,,Central Dauphin School District

Another example

75 Chenango Ave Clinton NY13323,,Clinton Central School District

You should put this in the address column

,75 Chenango Ave Clinton NY13323,Clinton Central School District

Return a csv
"""

# followed by 

"""
Ok. Now instead of moving, add a fourth column that says "address", "school_district", "None" or "drop"
"""

# it returns the following csv

'\nOk. Now instead of moving, add a fourth column that says "address", "school_district", "None" or "drop"\n'

In [6028]:
gpt_df = pd.read_csv("classified_test_by_gpt.csv")
gpt_df.sample(10)

,title_1,address,school_district,classification
341,NaN,NaN,West Allegheny School District,NaN
2266,NaN,"1801 Bushkill Drive  Easton, PA 18040-8186  610-250-2400",Easton Area School District,NaN
2757,Russell County Public Schools,"84 Lorraine C. Turner Drive | Lebanon, VA 24266 | (276) 889-6500 | f (276) 889-6508",NaN,school_district
175,John Glenn School Corporation,NaN,NaN,school_district
22,NaN,"150 Park Avenue, Amityville, New York, 11701",AMITYVILLE UNION FREE SCHOOL DISTRICT,NaN
852,NaN,"3555 North Calhoun Road | Brookfield, WI 53005 | Phone: 262.781.3030",School District of Elmbrook,NaN
2312,NaN,"102 School Lane | Blairsville, PA 15717 | Ph: 724-459-5500 | Fx: 724-459-9209",River Valley School District,NaN
1706,Greater Clark County Schools,"2112 Utica-Sellersburg Road | Jeffersonville, IN 47130 | 812-920-1079",NaN,school_district
356,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",NaN,school_district
2979,NaN,"6225 El Camino Real | Carlsbad, California 92009 | 760-331-5000 p | 760-331-5094 f",Carlsbad Unified School District,NaN


In [6029]:
# let's check its distribution
gpt_df["classification"].value_counts(dropna=False)

classification
NaN                2149
school_district    1704
address              53
Name: count, dtype: int64

In [6030]:
# interestingly, there are no drops
# let's look at the slogan row
gpt_df[gpt_df["title_1"]=="Quality to the Core"]

,title_1,address,school_district,classification
21,Quality to the Core,NaN,Central Dauphin School District,address


In [6031]:
# this is a pretty bad classification
# turns out GPT was using Python under the hood
# I asked GPT to classify manually instead
# here is its results
gpt_df = pd.read_csv("manual_classified_test.csv")
gpt_df.sample(10)

,title_1,address,school_district,classification
3165,Lake Wales Charter Schools,"130 E Central Avenue | Lake Wales, FL 33853",NaN,school_district
2000,NaN,"510 South Franklin Street | Farmington, MO 63640 | Ph: (573) 701-1300 | Fx: (573) 701-1309",Farmington R-7 School District,NaN
171,NaN,"125 S. Black Horse Pike | Mt. Ephraim, NJ 08059 | 856-931-7807",Mt. Ephraim School District,NaN
3550,NaN,NaN,Hazleton Area School District,NaN
2439,NaN,"10 Education Drive . Beacon, NY 12508 . Phone: (845) 838-6900",Beacon City School District,NaN
3585,Fairless Local Schools,NaN,NaN,school_district
3175,NaN,NaN,NaN,NaN
883,NaN,"780 Ocean Avenue | Bohemia, NY 11716 | Ph: (631) 244-2215 | Fx: (631) 589-0683",Connetquot Central School District,NaN
1485,NaN,"99 South Division | Cowley, WY 82420 | P: 307-548-2254 | F: 307-548-7610",Big Horn County School District #1,NaN
1183,Policy Manual and Board Meetings,NaN,NaN,school_district


In [6032]:
# still terrible, and I realized it still uses python under the hood
# I will start a new session and emphasize not to program
# let's check a few stats first

# number of NA title_1
print(df[df["title_1"].isna()].shape[0])

# number of none NA title_1
print(df[~df["title_1"].isna()].shape[0])

2149
1757


In [6033]:
df.columns

Index(['URL', 'title_1', 'home_website', 'address', 'school_district',
       'contains_phone_number_title_1', 'contains_phone_number_title_2',
       'phone'],
      dtype='object')

In [6034]:
df = df.drop(columns=["contains_phone_number_title_1","contains_phone_number_title_2"])

In [6035]:
# try again
df["index"] = df.index

In [6036]:
# o1 identified these as addresses
print(df.loc[df["index"].isin([155,242, 1343])])
# handle them manually
df.loc[df["URL"]=="https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public", "title_1"] = None

# drop the rest
df.loc[df["index"].isin([242, 1343]), "title_1"] = None

                                                          URL  \
155   https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public   
242          https://go.boarddocs.com/oh/swl/Board.nsf/Public   
1343        https://go.boarddocs.com/pa/karn/Board.nsf/Public   

                              title_1               home_website  \
155   75 Chenango Ave Clinton NY13323       https://www.ccs.edu/   
242          Southwest Licking LSD OH   http://www.swl.k12.oh.us   
1343             1446 Kittanning Pike  https://www.kcasdk12.org/   

                                                                       address  \
155                                                                       None   
242   Address: 927-A South Street, Pataskala, Ohio 43062 | Phone: 740-927-3941   
1343                                                     Karns City, PA  16041   

                      school_district phone  index  
155   Clinton Central School District  None    155  
242                            

In [6037]:
# output the rest
# use prompts/prompt_3.txt
df[~df["title_1"].isna()].to_csv("test.csv", columns=["title_1","index"],index=False)

In [6038]:
# get the o1 classification
gpt_df = pd.read_csv("gpt_results/results_10.csv")

# merge them
df = pd.merge(df, gpt_df, how="left", on="index")


In [6039]:
# get the missing ones
df[(~df["title_1"].isna()) & (df["classification"].isna()) ].to_csv("test-2.csv", columns=["title_1","index"],index=False)

In [6040]:
# sanity check
# all NAs should not have classifications
assert df[(df["title_1"].isna()) & (~df["classification"].isna()) ].shape[0] == 0

# all non-NAs should have classifications
assert df[(~df["title_1"].isna()) & (df["classification"].isna()) ].shape[0] == 0

# the number of classification should be the number of rows on the right
assert gpt_df.shape[0] == df[~df["classification"].isna()].shape[0]

In [6041]:
df["classification"].value_counts()

classification
school_district    1617
drop                 86
unsure               51
Name: count, dtype: int64

In [6042]:
# check the ones that are unsure
df[df["classification"]=="unsure"]

,URL,title_1,home_website,address,school_district,phone,index,classification
26,https://go.boarddocs.com/sd/meade/Board.nsf/Public,"Meade County, South Dakota",http://www.meadecounty.org/,"1300 Sherman Street Ste 222 | Sturgis, SD 57785",NaN,None,26,unsure
152,https://go.boarddocs.com/ks/kta/Board.nsf/Public,Kansas Turnpike Authority,http://www.ksturnpike.com/,"9401 E Kellogg | Wichita, KS 67207 | (316) 682-4537",NaN,None,152,unsure
165,https://go.boarddocs.com/il/vobil/Board.nsf/Public,"The Village of Bourbonnais, Illinois",http://www.villageofbourbonnais.com/,"600 Main St N.W. | Bourbonnais, IL 60914",NaN,None,165,unsure
269,https://go.boarddocs.com/il/naperville/Board.nsf/Public,Naperville Park District,https://www.napervilleparks.org/,"320. W. Jackson Ave. Naperville, IL 60540 | 630-848-5000",NaN,None,269,unsure
303,https://go.boarddocs.com/mn/mboa/Board.nsf/Public,Minnesota Board on Aging,http://www.mn.gov/board-on-aging,"540 Cedar Street St. Paul, MN 55155 | 1-800-882-6262",NaN,None,303,unsure
320,https://go.boarddocs.com/ca/phcd/Board.nsf/Public,Peninsula Health Care District,http://www.peninsulahealthcaredistrict.org,"1819 Trousdale Drive | Burlingame, CA 94010 | (650) 697-6900",NaN,None,320,unsure
609,https://go.boarddocs.com/il/ctpf/Board.nsf/Public,Public School Teachers' Pension and Retirement Fund of Chicago,http://www.ctpf.org,"425 S Financial Place, Suite 1400, Chicago, IL 60605 | 312-604-1400",NaN,None,609,unsure
620,https://go.boarddocs.com/mi/jackson/Board.nsf/Public,"Jackson County, Michigan",http://www.mijackson.org,"120 West Michigan | Jackson, MI 49201 | P: (517) 788-4335 | F: (517) 780-4755",NaN,None,620,unsure
653,https://go.boarddocs.com/fl/sefbhn/Board.nsf/Public,Southeast Florida Behavioral Health Network,http://sefbhn.org/,"8895 N Military Trail, Suite E-102 | Palm Beach Gardens, FL 33410 | (561) 203-2485",NaN,None,653,unsure
697,https://go.boarddocs.com/ok/plib/Board.nsf/Public,Pioneer Library System,http://www.pioneerlibrarysystem.org,"300 Norman Center Court | Norman, OK 73072 | 405.801.4500",NaN,None,697,unsure


In [6043]:
# the unsures all look like valid entities
# and their school_district field is empty
# we will classify them as school_district

df.loc[df["classification"]=="unsure", "classification"] = "school_district"

In [6044]:
# look at those with drops
df.loc[df["classification"]=="drop",:].sample(5)

,URL,title_1,home_website,address,school_district,phone,index,classification
136,https://go.boarddocs.com/pa/iroq/Board.nsf/Public,Board Meeting Documents and Policies,http://www.iroquoissd.org,None,IROQUOIS SCHOOL DISTRICT,None,136,drop
3637,https://go.boarddocs.com/mi/washisd/Board.nsf/Public,Neola Board Documents and Policies,https://washtenawisd.org,None,Washtenaw Intermediate School District,None,3637,drop
2966,https://go.boarddocs.com/wi/nles/Board.nsf/Public,District Policy Manual,www.northlandlakeschool.com,None,North Lakeland School District,None,2966,drop
3486,https://go.boarddocs.com/ia/bettcsd/Board.nsf/Public,Community - Lifelong Learning - Integrity- Empowerment,http://www.bettendorf.k12.ia.us,None,Bettendorf Community School District,None,3486,drop
2457,https://go.boarddocs.com/oh/gmsoh/Board.nsf/Public,eGovernance,http://www.gocruisers.org/default.aspx,None,NaN,None,2457,drop


In [6045]:
# drop the ones classified as drops
df.loc[df["classification"]=="drop", "title_1"] = None
df.loc[df["classification"]=="drop", "classification"] = None

In [6046]:
# look at those classified as school_district
df.loc[df["classification"]=="school_district",:].sample(5)

,URL,title_1,home_website,address,school_district,phone,index,classification
1644,https://go.boarddocs.com/oh/camr/Board.nsf/Public,Cambridge City Schools,http://cambridgecityschools.org/,None,NaN,None,1644,school_district
1974,https://go.boarddocs.com/il/district87/Board.nsf/Public,Bloomington Public Schools,http://www.district87.org/,"300 East Monroe | Bloomington, IL 61701 | Ph: 309-827-6031 | Fx: 309-827-5717",NaN,None,1974,school_district
1895,https://go.boarddocs.com/in/csf/Board.nsf/Public,Community Schools of Frankfort,http://frankfortschools.org,"2400 E. Wabash St. | Frankfort, IN 46041 | Ph: 765-654-5585 | fx: 765-659-6220",NaN,None,1895,school_district
3348,https://go.boarddocs.com/in/msdboone/Board.nsf/Public,MSD of Boone Township,http://www.hebronschools.k12.in.us,"307 S. Main St. Hebron, IN 46341 | 219-996-4771 (Opt #8)",NaN,None,3348,school_district
3483,https://go.boarddocs.com/ks/usd229/Board.nsf/Public,Blue Valley Schools,https://www.bluevalleyk12.org/,"15020 Metcalf, Overland Park, KS | (913) 239-4000",NaN,None,3483,school_district


In [6047]:
# move the others to school_district
df.loc[df["classification"]=="school_district", "school_district"] = df["title_1"]
df.loc[df["classification"]=="school_district", "title_1"] = None
df.loc[df["classification"]=="school_district", "classification"] = None

# assert no others
assert df[~df["classification"].isna()].shape[0] == 0
assert df[~df["title_1"].isna()].shape[0] == 0

In [6048]:
# delete these columns
df = df.drop(["classification","title_1"], axis=1)

In [6049]:
# we also try to get the phone number from the address
df['address_contains_phone_number'] = df['address'].apply(contains_phone_number)
df[df["address_contains_phone_number"]].sample(5)

,URL,home_website,address,school_district,phone,index,address_contains_phone_number
2258,https://go.boarddocs.com/az/balsz/Board.nsf/Public,http://www.balsz.org,"4825 E Roosevelt | Phoenix, AZ 85008 | p 602.629.6400 | f 602.629.6470",Balsz Elementary School District #31,None,2258,True
1686,https://go.boarddocs.com/va/mcpsva/Board.nsf/Public,http://www.mcpsva.org/,"8700 Centreville Rd. Suite 400 | Manassas, VA 20110 | 571-377-6000 | f 703-257-8801",Manassas City Public Schools,None,1686,True
2371,https://go.boarddocs.com/wi/whsd/Board.nsf/Public,http://www.wisheights.k12.wi.us/,"10173 US Highway 14 | Mazomanie, WI 53560 | (608) 767-2595 | f (608) 767-3579",Wisconsin Heights School District,None,2371,True
1855,https://go.boarddocs.com/ks/comks/Board.nsf/Public,http://www.mcpcity.com/,"400 E Kansas Avenue | P.O. Box 1008 | McPherson, KS 67460 | 620-245-2535 | 620-245-2549",City of McPherson Kansas,None,1855,True
3122,https://go.boarddocs.com/ny/nfcsd/Board.nsf/Public,http://www.nfschools.net,"City School District of the City of Niagara Falls| 630 - 66th Street, Niagara Falls, NY 14304 | Phone: 716-286-4211",NIAGARA FALLS BOARD OF EDUCATION,None,3122,True


In [6050]:
df[df["address_contains_phone_number"]].shape

(2185, 7)

In [6051]:
# extract it
# but before that, check if conflicts
df[df["address_contains_phone_number"] & (~df["phone"].isna())].shape

(0, 7)

In [6052]:
# great! no conflicts
phone_pattern = re.compile(r'(\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b)')
df["phone_extracted_from_address"] = df["address"].str.extract(phone_pattern)

In [6053]:
df[~df["phone_extracted_from_address"].isna()].sample(5)

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address
3811,https://go.boarddocs.com/vsba/charles/Board.nsf/Public,http://www.ccps.net,"10035 Courthouse Road | Charles City, VA 23030 | 804.652.4612 | f 804.829-2363",Charles City County Public Schools,None,3811,True,804.652.4612
692,https://go.boarddocs.com/oh/granville/Board.nsf/Public,https://www.granvilleschools.org/,"130 North Granger Street | Granville, OH 43023 | 740-587-8101",Granville Exempted Village School District,None,692,True,740-587-8101
2541,https://go.boarddocs.com/wi/mmsd/Board.nsf/Public,https://www.madison.k12.wi.us/,"545 W. Dayton St. | Madison, Wisconsin 53703 | p (608) 663-1659 |",Madison Metropolitan School District,None,2541,True,(608) 663-1659
3252,https://go.boarddocs.com/ks/usd408/Board.nsf/Public,http://www.usd408.com/,"101 N. Thorp | Marion, KS 66861 | 620-382-2117","Marion-Florence USD 408, Kansas",None,3252,True,620-382-2117
1710,https://go.boarddocs.com/oh/acesc/Board.nsf/Public,http://www.ashtabulaesc.org,"2630 West 13th Street, Suite A | Ashtabula, OH 44004 | Phone (440) 576-9023",Ashtabula County Educational Service Center,None,1710,True,(440) 576-9023


In [6054]:
# double check no clashes
df[(~df["phone_extracted_from_address"].isna()) & (~df["phone"].isna())]

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address


In [6055]:
# write in
df.loc[~df["phone_extracted_from_address"].isna(), "phone"] = df["phone_extracted_from_address"]
df.loc[~df["phone_extracted_from_address"].isna(), "phone_extracted_from_address"] = None

In [6056]:
# let's clean it
# extract all numbers and make sure there are 10

df['extracted_phone_numbers'] = None
df['extracted_phone_numbers'] = df['phone'].astype(str).str.replace(r'[^\d]', '', regex=True)

In [6057]:
# check their length
df["phone_length"] = df["extracted_phone_numbers"].apply(lambda x: len(x))
df["phone_length"].value_counts()

phone_length
10    2216
0     1684
20       4
12       1
30       1
Name: count, dtype: int64

In [6058]:
# check those that are not 10 or 0
df.loc[(df["phone_length"]!=10) & (df["phone_length"]!=0)]

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address,extracted_phone_numbers,phone_length
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,http://www.triton.k12.in.us,None,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,624,False,NaN,574342225512,12
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,Phone: 715-257-7511 Fax: 715-257-7502,1199,False,NaN,71525775117152577502,20
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,2173,False,NaN,913592727291359272009135927270,30
2538,https://go.boarddocs.com/wi/campsd/Board.nsf/Public,www.csd.k12.wi.us,"327 N. Fond du Lac Ave. Campbellsport, WI 53010",NaN,(920) 533-8381 | Fax (920) 533 -5726,2538,False,NaN,92053383819205335726,20
2586,https://go.boarddocs.com/oh/cevsdoh/Board.nsf/Public,www.carrollton.k12.oh.us,"Carrollton Exempted Village School District, - 205 Scio Road S.W., - Carrollton, OH 44615",NaN,"Phone: 330-627-2181, - Fax: 330-627-2182",2586,False,NaN,33062721813306272182,20
2811,https://go.boarddocs.com/id/nsd131/Board.nsf/Public,http://www.nsd131.org,"619 S. Canyon St | Nampa, ID 83686",NaN,(208) 468-4600 Fax: (208) 468-4638,2811,False,NaN,20846846002084684638,20


In [6059]:
# ok, we can just take the top 10 numbers
df["extracted_phone_numbers"] = df["extracted_phone_numbers"].apply(lambda x: x[:10] if x else None)

In [6060]:
# put them in a nice format
def format_phone_numbers(number):
    if not number:
        return None
    # Convert number to string for easy slicing
    num_str = str(number)
    if len(num_str) == 10:  # Ensure it's a 10-digit number
        formatted = f"({num_str[:3]}) {num_str[3:6]}-{num_str[6:]}"
        return formatted
    else:
        raise Exception("Invalid number")

df["phone"] = df["extracted_phone_numbers"].apply(format_phone_numbers)

In [6061]:
# clean up the columns
df = df.loc[:,["URL","home_website","address","school_district","phone"]]
df.sample(5)

,URL,home_website,address,school_district,phone
1545,https://go.boarddocs.com/il/est189/Board.nsf/Public,http://www.estl189.com/,"1005 State St | East Saint Louis, IL 62201-1907 | Ph: (618) 646-3001 | Fx: (618) 583-8295",East St. Louis School District 189,(618) 646-3001
363,https://go.boarddocs.com/ar/searcy/Board.nsf/Public,https://www.searcyschools.org/,"801 N. Elm St. Searcy, Arkansas AR 72143",Searcy School District,None
2580,https://go.boarddocs.com/pa/sfay/Board.nsf/Public,NaN,None,South Fayette Township School District,None
516,https://go.boarddocs.com/mi/gbacademy/Board.nsf/Public,https://gbacademy.org/,"5135 E. Hill Rd., Grand Blanc, MI 48439 Phone: 810-953-3140",Grand Blanc Academy,(810) 953-3140
1367,https://go.boarddocs.com/az/cvit/Board.nsf/Public,https://www.cvit81.org/,"1500 W. Panther Dr., Bld 3, Rm 206, Superior, AZ 85173 | (928) 542.9640 Mailing Address PO Box 176, Superior, AZ 85173",Cobre Valley Institute of Technology,(928) 542-9640


In [6062]:
# finally, add a column indicate if the row is likely an actual school_district
# here is Tom's quote
"""
However, please add a column to the spreadsheet noting whether the words “school”, “academy”, “district” or “education” appear in the “address” or “school_district” columns.   That way, we can go through later and weed out the other organizations.
"""

'\nHowever, please add a column to the spreadsheet noting whether the words “school”, “academy”, “district” or “education” appear in the “address” or “school_district” columns.   That way, we can go through later and weed out the other organizations.\n'

In [6063]:
likely_school_district_pattern = r"school|academy|district|education"
df["contain_school_district_keywords"] = df["address"].str.contains(likely_school_district_pattern, case=False, regex=True, na=False) | df["school_district"].str.contains(likely_school_district_pattern, case=False, regex=True, na=False) 

In [6064]:
# get the stats
df["contain_school_district_keywords"].value_counts()

contain_school_district_keywords
True     3407
False     499
Name: count, dtype: int64

In [6065]:
# output the result
df.to_csv("release/deliverable_1.csv", index=False, columns=["URL","school_district","address","home_website","phone"])